In [ ]:
import pandas as pd
import yaml

with open("../../config.yaml", "r") as f:
    cfg = yaml.safe_load(f)


df = pd.read_csv(cfg["house_prices_hw"]["train_dataset"])
df_test = pd.read_csv(cfg["house_prices_hw"]["test_dataset"])

In [ ]:
df.columns

Преобразуем целевую переменную SalePrice с помощью логарифма, чтобы уменьшить влияние выбросов


In [ ]:
import numpy as np

df_copy = df.copy()
y = np.log1p(df_copy["SalePrice"])

X = df_copy.drop(columns=["SalePrice"])
X_test = df_test.copy()

Определяем числовые и категориальные признаки


In [ ]:
num_cols = X.select_dtypes(include=["number"]).columns.tolist()
cat_cols = X.select_dtypes(include=["object"]).columns.tolist()

Создаем конвейеры обработки данных\
Для числовых признаков пропущенные значения заполняются средним значением\
Для категориальных признаков значения заполняются наиболее частым значением\
Переменные категориальных признаков кодируются в числовой формат OneHotEncoder


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder


num_pipe = SimpleImputer(strategy="mean")

cat_pipe = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="most_frequent")),
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
    ]
)

Создаем общий обработчик числовых и категориальных признаков


In [ ]:
from sklearn.compose import ColumnTransformer


transformer = ColumnTransformer(
    [("num", num_pipe, num_cols), ("cat", cat_pipe, cat_cols)]
)

Создаем основной пайплайн (предобработка и модель Ridge-регрессии)


In [ ]:
from sklearn.linear_model import Ridge


model = Pipeline([("transform", transformer), ("ridge", Ridge())])

Подбираем параметр alpha с помощью кросс-валидации\
Используем отрицательное среднеквадратическое отклонение\
GridSearchCV максимизирует метрику, но RMSE - ошибка, значит метрику необходимо минимизировать


In [ ]:
from sklearn.model_selection import GridSearchCV


params = {"ridge__alpha": np.logspace(-4, 4, 10)}
search = GridSearchCV(model, params, cv=4, scoring="neg_root_mean_squared_error")
search.fit(X, y)
best_model = search.best_estimator_

Оценим качество модели с помощью RMSE на кросс-валидации


In [ ]:
from sklearn.model_selection import cross_val_score


rmse = cross_val_score(
    best_model, X, y, cv=4, scoring="neg_root_mean_squared_error"
).mean()

print("mean RMSE: ", rmse)

Предсказываем результаты на тестовых данных\
Результат преобразуется обратно из логарифмической шкалы, округляется и приводится к числовому типу


In [ ]:
pred = best_model.predict(X_test)
result = np.expm1(pred).round().astype(int)

pd.DataFrame({"Id": df_test["Id"], "SalePrice": result}).to_csv(
    "output.csv", index=False
)